<h1 align="center">Scientific Programming in Python</h1>
<h2 align="center">Topic 2: NumPy and Efficient Numerical Programming</h2> 

_Notebook created by Martín Villanueva - `martin.villanueva@usm.cl` - DI UTFSM - April 2017._

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

## Table of Contents
* [1.- Useful Magics](#magics)
* [2.- Basic NumPy Operations](#basic_numpy)
* [3.- Internals of NumPy](#internals)
* [4.- Efficient Programming in NumPy](#efficient)
* [5.- Vectorized Algorithms in NumPy](#vectorized)
* [6.- Bonus ](#bonus)

<div id='magics' />
## Useful magic built-in functions for time-profiling

* **time**: See how long it takes a code to run.

In [2]:
%time {1+2 for i in range(10000)}

CPU times: user 928 µs, sys: 1 µs, total: 929 µs
Wall time: 938 µs


{3}


* **timeit**: 
    * _See how long a code takes to run averaged over multiple runs_.  
    * It will limit the number of runs depending on how long the script takes to execute.  
    * Provide an accurate time calculation by reducing the impact of startup or shutdown costs on the time calculation by executing the code repeatedly.

In [35]:
%timeit [1+2 for i in range(10000)]

1000 loops, best of 3: 446 µs per loop


In [36]:
%timeit -n 100 [1+2 for i in range(10000)]

100 loops, best of 3: 435 µs per loop


In [37]:
%%timeit
for i in range(10000):
    1+2

1000 loops, best of 3: 367 µs per loop


* **prun**: See how long it took each function in a script to run.

In [3]:
from time import sleep

def foo(): sleep(1)
    
def bar(): sleep(2)
    
def baz(): foo(),bar()

In [4]:
%prun baz()

<div id='basic_numpy' />
## Basic NumPy operations

The reasons of why you should use NumPy instead of any other __iterable object_ in Python are:
* NumPy provides an ndarray structure for storing numerical data __in a contiguous way__.
* Also implements fast __mathematical operations__ on ndarrays, that exploit this contiguity.
* __Brevity of the syntax for array operations__. A language like C or Java would require us to write a loop for a matrix operation as simple as C=A+B.

### Creating Arrays

There are several NumPy functions for creating common types of arrays. Below is a list of the most common used:

In [9]:
# Arrays of zeros: np.zeros(shape)
print("Zeros:")
print( np.zeros((3,3)) )

# Arrays of ones: np.ones(shape)
print("\nOnes:")
print( np.ones((3,3)) )

# Empty array: np.empty(shape)
print("\nEmpty:")
print( np.empty((3,3)) )

# Range of values: np.range(start, stop, step)
print("\nRange:")
print( np.arange(0., 10., 1.) )

# Regular grid: np.linspace(start, end, n_values)
print("\nRegular grid:")
print( np.linspace(0., 1., 9) )

# Random secuences: np.random
print("\nRandom sequences:")
print( np.random.uniform(10, size=6) )

# Array constructor: np.array( python_iterable )
print("\nArray constructor")
print( np.array([2, 3, 5, 10, -1]) )


print( 10*np.random.random((5,5)) )

Zeros:
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]

Ones:
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]

Empty:
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]

Range:
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]

Regular grid:
[ 0.     0.125  0.25   0.375  0.5    0.625  0.75   0.875  1.   ]

Random sequences:
[ 7.0107102   7.74748288  7.52060647  4.51119533  1.20685707  2.26318781]

Array constructor
[ 2  3  5 10 -1]
[[ 1.34564829  0.73186144  7.10479207  3.42345723  6.75970902]
 [ 7.62384409  8.64156341  2.6413028   3.18163139  1.37541434]
 [ 9.17431802  6.62975508  8.59352212  0.17215126  1.42169191]
 [ 2.32145598  3.55130399  8.19111289  3.65115003  8.38451744]
 [ 3.38776782  1.61687056  7.9962429   6.49598812  4.56252822]]


### Basic Mathematical operations

Most of the operations performed in NumPy are handled __element-wise__, i.e, computing C = A + B will translates into $C[i,j] = A[i,j] + B[i,j]$. (The exception is __broadcasting__, and will be explained soon).

Below is a list with the most common used mathematical operations. For a comprehensive list se here: [NumPy mathematical functions](https://docs.scipy.org/doc/numpy/reference/routines.math.html).

In [10]:
# first we create two random arrays:
A = np.random.random((5,5))
B = np.random.random((5,5))

# sum
print("Sum:")
print( A+B )

# subtraction
print("\nSubtraction")
print( A-B )

# product
print("\nProduct")
print( A*B )

# matricial product
print("\nMatricial Product")
print( np.dot(A,B) )

# power
print("\n Power")
print( A**2 )

# Some common mathematical functions
print("\n np.exp()")
print( np.exp(A) )
print("\n np.sin()")
print( np.sin(A) )
print("\n np.cos()")
print( np.cos(A))
print("\n np.tan()")
print( np.tan(A) )

Sum:
[[ 1.53652038  0.52007791  1.04351631  1.00065342  1.19390632]
 [ 0.60426392  1.45132764  0.75379862  1.85485639  1.5179623 ]
 [ 0.55636747  1.03083353  1.44038638  1.23211128  1.1853136 ]
 [ 1.15602214  0.85536963  1.52813733  1.02165423  1.16355959]
 [ 0.0894018   0.94960096  1.61486698  0.98062422  0.70217686]]

Subtraction
[[ 0.25289158 -0.14148365 -0.04121787  0.90758554  0.78531883]
 [ 0.46307677 -0.33495048 -0.22768754 -0.00167064  0.42308183]
 [ 0.43107209 -0.32259019 -0.18307721 -0.69396348 -0.07466717]
 [ 0.26040146 -0.80762233 -0.44445835  0.39698819  0.56093015]
 [ 0.07302493  0.50360849  0.31125776 -0.75442611 -0.65776296]]

Product
[[  5.74235182e-01   6.26158532e-02   2.71806845e-01   4.43989409e-02
    2.02171658e-01]
 [  3.76736972e-02   4.98540027e-01   1.29092684e-01   8.60122356e-01
    5.31302825e-01]
 [  3.09304036e-02   2.39638336e-01   5.10298915e-01   2.59128225e-01
    3.49848288e-01]
 [  3.17144568e-01   1.98508435e-02   5.34415118e-01   2.21544437e-01
 

### Boolean operations

Comparisons in NumPy work exaclty the same way as mathematical operations, i.e, __element wise!__. Let's see some examples:

In [11]:
# Creating two 2d-arrays
A = np.array( [[1, 2, 3], [2, 3, 5], [1, 9, 6]] )
B = np.array( [[1, 2, 3], [3, 5, 5], [0, 8, 5]] )

print("A > B:")
print( A > B )

print("\nA =< B:")
print( A <= B )

print("\n A==B:")
print( A==B )

print("\n A!=B:")
print( A!=B )

# Creating two 2d boolean arrays
C = A==B
D = A>=B

print("\n A and B:")
print( C & D)
print( np.logical_and(C,D) )

print("\n A or B:")
print( C | D)
print( np.logical_or(C,D) )

print("\n not A:")
print( ~C )
print( np.logical_not(C))

A > B:
[[False False False]
 [False False False]
 [ True  True  True]]

A =< B:
[[ True  True  True]
 [ True  True  True]
 [False False False]]

 A==B:
[[ True  True  True]
 [False False  True]
 [False False False]]

 A!=B:
[[False False False]
 [ True  True False]
 [ True  True  True]]

 A and B:
[[ True  True  True]
 [False False  True]
 [False False False]]

 A or B:
[[ True  True  True]
 [False False  True]
 [ True  True  True]]

 not A:
[[False False False]
 [ True  True False]
 [ True  True  True]]


<div id='internals' />
## Internals of NumPy

### The `numpy.ndarray` structure

The `ndarray` is the NumPy object that let us create __$N$-dimensional arrays__. It is essentially defined by:
1. A number of __dimensions__
2. a __shape__
3. __strides__
4. __data type__ or __dtpe__
5. The __data buffer__.

<img src='data/ndarray.png' style="width: 500px;">

In [12]:
# Lets create a random array
A = np.random.random((5,5))

print("Dims: ")
print(A.ndim)

print("\nShape: ")
print(A.shape)

print("\nStrides: ")
print(A.strides)

print("\nData type: ")
print(A.dtype)

Dims: 
2

Shape: 
(5, 5)

Strides: 
(40, 8)

Data type: 
float64


### How an `ndarray` is stored in memory?

When there is more than one dimension, there are two ways of storing the elements in the memory block:
1. Elements can be stored in __row-major order__ (also known as __C-order__) or,
2. In __column-major order__ (also known as __Fortran-order__).

<img src='data/ndarray_storage.png' style="width: 800px;">


### What are the `strides`?

NumPy uses the notion of strides to convert between a multidimensional index and the memory location of the underlying (1D) sequence of elements. 

For example, the mapping between array[i,j] and the corresponding address the byte is:
* `offset = array.strides[0] * i1 + array.strides[1] * i2`
* `address = base + offset`
where base is the address of the first byte (`array[0,0]`).

In [13]:
# Lets create a random array in C-order
A = np.random.random((5,2))
print("C strides:")
print(A.strides)

# Lets create a random array in F-order
B = np.asfortranarray(A)
print("\nF strides:")
print(B.strides)

C strides:
(16, 8)

F strides:
(8, 40)


### An interesting result

In [14]:
X = np.random.random((5000,5000))

In [15]:
%timeit X[0,:].sum()

The slowest run took 41.16 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.31 µs per loop


In [16]:
%timeit X[:,0].sum()

The slowest run took 43.79 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 48.7 µs per loop


### Why Python is so slow?

* ** Python is Dynamically Typed rather than Statically Typed.** What this means is that at the time the program executes, the interpreter doesn't know the type of the variables that are defined. <img src='data/cint_vs_pyint.png' style="width: 300px;">
* **Python is interpreted rather than compiled.** A smart compiler can look ahead and optimize for repeated or unneeded operations, which can result in speed-ups. 
* **Python's object model can lead to inefficient memory access.** A NumPy array in its simplest form is a Python object build around a C array. That is, it has a pointer to a contiguous data buffer of values. A Python list, on the other hand, has a pointer to a contiguous buffer of pointers, each of which points to a Python object which in turn has references to its data (in this case, integers). <img src='data/array_vs_list.png' style="width: 500px;">

### Why Numpy is so fast?

* Computations follow the **Single Instruction Multiple Data** (SIMD) paradigm. So that NumPy can take advantage of vectorized instructions on modern CPUs, like Intel's SSE and AVX, AMD's XOP.
<img src='data/devectorized.png' style="width: 350px;", caption='asdf'>
<img src='data/vectorized.png' style="width: 350px;">
* A NumPy array is described by metadata (number of dimensions, shape, data type, strides, and so on) and the data (which is stored in a **homogeneous and contiguous** blocks of memory).
* **Array computations** can be written very efficiently in a low-level language like C (and a large part of NumPy is actually written in C). Aditionally many internal methods and functions are linked to highly optimized linear algebra libraries like **BLAS** (Basic Linear Algebra Subprograms) and **LAPACK** (Linear Algebra PACKage).
* **Spatial locality** in memory access patterns results in significant performance gains, notably thanks to the CPU cache.  Indeed, the cache loads bytes in chunks from RAM to the CPU registers.

### Lets see the problem...

Consider the problem of calculating the squared norm of a vector ($\displaystyle || \mathbf{v} ||^2 = \mathbf{v} \cdot \mathbf{v}$) with the following 4 implementations:

In [17]:
#Python Lists implementation
def norm_square_list(vector):
    norm = 0
    for v in vector:
        norm += v*v
    return norm

#Naive NumPy implementation
def norm_square_array(vector):
    norm = 0
    for v in vector:
        norm += v*v
    return norm

#Vectorized NumPy implementation
def norm_square_numpy(vector):
    return np.sum(vector * vector)

#Clever NumPy implementation
def norm_square_dot(vector):
    return np.dot(vector, vector)

#Vector to use - dimension 10^6
vector = range(1000000)
npvector = np.array(vector)

In [18]:
#Timing the list implementation
%timeit norm_square_list(vector)

10 loops, best of 3: 128 ms per loop


In [19]:
#Timing the naive array implementation
%timeit norm_square_array(npvector)

1 loop, best of 3: 197 ms per loop


In [20]:
#Timing the NumPy-vectorized implementation
%timeit norm_square_numpy(npvector)

100 loops, best of 3: 3.86 ms per loop


In [21]:
#Timing the clever NumPy-vectorized implementation
%timeit norm_square_dot(npvector)

1000 loops, best of 3: 812 µs per loop


As you can see, some interesting things have happened:

* The naive NumPy implementation, which iterates over data is actually __slower__ than simply using a list. This is because the *array* stores a  very low-level representation of the numbers it stores , and this **must be converted** into Python-compatible objects before being returned to user, causing this extra overhead each time you index an *array*.
* `norm_square_numpy` is slower than the clever NumPy implementation because two reasons:
    1. There is time spend in allocating memory for storing the temporary result `(vector x vector)` and
    2. This creates two **implied loops**, one to do the multiplication and one to do the sum.
* The clever implementation uses *np.dot()* NumPy function which has no need to store intermediate results, and iterates just one time (but at C speed).

<div id='efficient' />
## Eficient programming with NumPy

### In-place and implicit copy operations

Prefer in-place over implicit-copy operations whenever possible. This will save memory (less work to garbage collector) and performs faster.

In [22]:
def id(x):
    """
    This function returns the memory
    block address of an array.
    """
    return x.__array_interface__['data'][0]

Array computations can involve **in-place** operations (first example below: the array is modified) or **implicit-copy** operations (second example: a new array is created).

In [23]:
a = np.zeros(10); aid = id(a)

In [24]:
# in-place operation
a *= 2; id(a) == aid

True

In [25]:
# implicit-copy operation
a = a * 2; id(a) == aid

False

Be sure to choose the type of operation you actually need. Implicit-copy operations are **slower!**

In [26]:
%%timeit 
a = np.ones(100000000)
a *= 2

1 loop, best of 3: 492 ms per loop


In [27]:
%%timeit
a = np.ones(100000000)
b = a * 2

1 loop, best of 3: 846 ms per loop


### Efficient memory access

We have basically three alternatives to access arrays without loops:

1. **array slicing**
2. **boolean masks**
3. **fancy indexing**. 

__Note.__ If you find yourself looping over indices to select items on which operation is performed, it can probably be done more efficiently with one of these techniques!

In [28]:
m, n = 1000000, 100
a = np.random.random_sample((m, n))
index = np.arange(0, m, 10)

In [29]:
#fancy indexing - indexing with lists
%timeit a[index,:]

10 loops, best of 3: 44.7 ms per loop


In [30]:
#memory slice - memory views
%timeit a[::10]

The slowest run took 23.64 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 267 ns per loop


*  Array slices are implemented as __memory views__, i.e, refer to the original data buffer of an array, but with different __offsets__, __shapes__ and __strides__.
* Array views should be used whenever possible, but one needs to be careful about the fact that views refer to the original data buffer.
* Fancy indexing is several orders of magnitude slower as it involves copying a large array.

Another useful indexing technique is the __mask of booleans__. Lets suppose we want to get all the elements on array with value less than 0.5

In [31]:
def naive_indexing(vect):
    ret = list()
    for val in vect:
        if val < 0.5: ret.append(val)
    return np.array(ret)

#data to occupy and mask of booleans
vect = np.random.random_sample(1000000)
mask = vect < 0.5
mask

array([ True, False, False, ...,  True,  True, False], dtype=bool)

In [32]:
#naive indexing
%timeit naive_indexing(vect)

1 loop, best of 3: 229 ms per loop


In [33]:
#mask indexing
%timeit vect[mask]

100 loops, best of 3: 7.77 ms per loop


In [34]:
#'improved' mask indexing
%timeit np.compress(mask, vect)

100 loops, best of 3: 7.81 ms per loop


### Broadcasting

There is no need to always reshape arrays before operate on them. This useful feature implemented in NumPy arrays is called **Broadcasting rules.** In the visualization below, the extra memory indicated by the dotted boxes is never allocated, but it can be convenient to think about the operations as if it is.
<img src='data/broadcasting.png' style="width: 600px;">

__How it works:__ The two arrays to be operated must match in at least one dimension. Then the array with less dimensions will __logically__ extended to match the dimensions of the other

In [31]:
# array([0,1,2]) + 5
np.arange(3) + 5

array([5, 6, 7])

In [32]:
# array([[1, 1 ,1], [1, 1, 1], [1, 1, 1]]) + array([0, 1, 2])
np.ones((3,3)) + np.arange(3)

array([[ 1.,  2.,  3.],
       [ 1.,  2.,  3.],
       [ 1.,  2.,  3.]])

In [33]:
# array([[0], [1], [2]]) + array([0, 1 ,2])
np.arange(3).reshape((3,1)) + np.arange(3)

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

<div id='vectorized' />
## Vectorized Algorithms with NumPy

**Vectorization:** *The process of converting a scalar implementation, which processes a single pair of operands at a time, to a vector implementation, which processes one operation on multiple pairs of operands at once.*

### Example: Gram-Shmidt Orthogonalization

**The problem**: Given a matrix $Q_{m\times n}$ with ($m>n$), find a vector $v$ orthogonal to the column space of $Q$. 
<img src='data/orthogonalization.jpg' style="width: 400px;">

In [47]:
def naive_orth(Q,v):
    m,n = Q.shape
    for j in range(n):
        v -= np.dot(Q[:,j],v)*Q[:,j]
    return v
    
def vectorized_orth(Q,v):
    proy = np.dot(Q.transpose(),v)
    # v -= (proy*Q).sum(axis=1)
    v -= np.dot(Q,proy)
    return v

In [48]:
# Let's generate a random unitary matrix
# Q unitary matrix, dimensions 100000 x1000
m,n = 10000,100
A = 10 * np.random.random((m,n))
Q,R = np.linalg.qr(A, mode='reduced')
del R

# v will be the starting vector for orthogonalization
v = np.random.random(m)
v1 = v.copy()
v2 = v.copy()

In [49]:
%timeit naive_orth(Q,v1)

100 loops, best of 3: 12 ms per loop


In [50]:
%timeit vectorized_orth(Q,v2)

1000 loops, best of 3: 889 µs per loop


<div id='bonus' />
## Bonus: Useful libraries based in NumPy

### numpy.linalg (Numpy's submodule)

In [44]:
A = np.random.random((100,100))
b = np.random.random(100)
c = np.random.random(100)

### Matrix power ###
np.linalg.matrix_power(A,3)

### Cholesky decomposition ###
#np.linalg.cholesky(A) #A must be positive definite

### QR decomposition ###
np.linalg.qr(A, mode='reduced')

### SVD decomposition ###
np.linalg.svd(A, full_matrices=False)

### Eigenvectors ###
np.linalg.eig(A)

### Eigevalues ###
np.linalg.eigvals(A)

### Matrix or vector norm ###
np.linalg.norm(A, ord='fro')

### Condition number ###
np.linalg.cond(A, p=np.inf)

### Determinant ###
np.linalg.det(A)

### Linear solver Ax=b ###
np.linalg.solve(A,b)

### Least Squares Ax=b (over-determined) ###
np.linalg.lstsq(A,b)

### Inverse ###
np.linalg.inv(A)

### Pseudo-Inverse ###
np.linalg.pinv(A)

### and many more...
del A,b,c

### scipy.optimize(SciPy's submodule)

* bisect(f, a, b[, args, xtol, rtol, maxiter, ...]) -  	Find root of a function within an interval.
* newton(func, x0[, fprime, args, tol, ...]) -	Find a zero using the Newton-Raphson or secant method.
* fixed_point(func, x0[, args, xtol, maxiter]) -	Find a fixed point of the function.
* root(fun, x0[, args, method, jac, tol, ...]) -	Find a root of a vector function.
* fsolve(func, x0[, args, fprime, ...]) -	Find the roots of a function.

### scipy.integrate (Scipy's submodule)

* cumtrapz(y[, x, dx, axis, initial]) -	Cumulatively integrate y(x) using the composite trapezoidal rule.
* simps(y[, x, dx, axis, even]) -	Integrate y(x) using samples along the given axis and the composite Simpson’s rule.
* fixed_quad(func, a, b[, args, n]) -	Compute a definite integral using fixed-order Gaussian quadrature.
* quadrature(func, a, b[, args, tol, rtol, ...]) -	Compute a definite integral using fixed-tolerance Gaussian quadrature.
* odeint(func, y0, t[, args, Dfun, col_deriv, ...]) -	Integrate a system of ordinary differential equations.

## References

* [*IPython notebook oficial documentation*](http://ipython.readthedocs.org/en/stable/)
* [*Time and memory Profiling in IPython*](http://pynash.org/2013/03/06/timing-and-profiling.html)
* [*Getting the best performance out of NumPy*](http://ipython-books.github.io/featured-01/)
* [*Why Python is slow*](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/)
* [*High Performance Python: Practical Performant Programming for Humans*](https://books.google.cl/books?id=bIZaBAAAQBAJ&pg=PA115&lpg=PA115&dq#v=onepage&q&f=false)